# Assignment: Linear and logistic regression
The assignment consists of constructing two separate models for predicting the real estate prices in the dataset: one with linear and one with logistic regression.

1. Linear regression model: construct a linear regression model for predicting the continuous target variable "Y house price of unit area" in the dataset.

2. Logistic regression model: convert the target variable into a binary-valued one according to whether the original target value is above or below the average house price of unit area (within the training set samples), and construct a binary classifier for predicting its value with logistic regression.

Both models should be validated, with appropriate metrics presented and discussed.

In [5]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
real_estate_valuation = fetch_ucirepo(id=477)

# data (as pandas dataframes)
X = real_estate_valuation.data.features
y = real_estate_valuation.data.targets

# variable information
print (real_estate_valuation.data)


{'ids':       No
0      1
1      2
2      3
3      4
4      5
..   ...
409  410
410  411
411  412
412  413
413  414

[414 rows x 1 columns], 'features':      X1 transaction date  X2 house age  \
0               2012.917          32.0   
1               2012.917          19.5   
2               2013.583          13.3   
3               2013.500          13.3   
4               2012.833           5.0   
..                   ...           ...   
409             2013.000          13.7   
410             2012.667           5.6   
411             2013.250          18.8   
412             2013.000           8.1   
413             2013.500           6.5   

     X3 distance to the nearest MRT station  X4 number of convenience stores  \
0                                  84.87882                               10   
1                                 306.59470                                9   
2                                 561.98450                                5   
3                     

## Business Understanding
- The first phase is to understand the business problem that needs to be solved.
- What is the goal of the analysis?
- What are the requirements and constraints? What is the expected outcome?

## Data Understanding
- The second phase is to collect and explore the data.
- What data is available?
- What are the characteristics of the data (variable types, value distributions etc.)?
- Are there any quality issues with the data (missing values, outliers, nonsensical values)?

## Data Preparation
- The third phase is to preprocess the data.
- This includes cleaning the data, transforming the data, and selecting the relevant features.
- These steps should be documented in such detail that they can be reproduced later.

## Modeling
- The fourth phase is to choose a machine learning method and train the model.
- This phase also includes the validation of the model.
- Documentation needs include:
-- which method was used, which parameters were used, what was the performance of the model?

## Evaluation
- The fifth phase is to evaluate the model.
- How well does the model perform?
- Does it meet the business requirements?

## Deployment
- The final phase is to deploy the model.
- How will the model be used in practice?
- How will the results be communicated?
- This phase may involve creating a recommendation of how to use the model in practice, or what to do next.